In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession.builder.appName("exercice").getOrCreate()

23/08/15 18:20:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [26]:
from pyspark.ml.regression import LinearRegression

In [29]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [30]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [31]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

# SET UP THE INPUTS AND OUTPUTS OF THE ML MODEL : THE MOST IMPORT AND HARD THING TO DO

In [32]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# WHAT ARE WE GOING TO DO ?
*   Convert all the numerical columns (num data)
*   so these are the **features**
    - |-- Avg Session Length: double (nullable = true)
    - |-- Time on App: double (nullable = true)
    - |-- Time on Website: double (nullable = true)
    - |-- Length of Membership: double (nullable = true)
*   and this is the **label** to predict 
    - |-- Yearly Amount Spent: double (nullable = true)

# LETS SEE HOW TO USE **VECTORASSEMBLER**

In [33]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [34]:
# creating an assembler object :
assembler = VectorAssembler(inputCols=[ 'Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

## EXPLICATION : 
what this function will do is to grab all of the input columns that we gave in **inputCols** and transform it to a single output which we called **features**

# NEXT STEP : **Transform the data**

In [35]:
output = assembler.transform(data)

In [36]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [38]:
final_data = output.select('features','Yearly Amount Spent')

In [39]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

# GOOD JOB !!!

# Now we are going to do the **SPLIT TRAIN TEST**

In [40]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [41]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [42]:
lr_model = lr.fit(train_data)

23/08/15 18:38:49 WARN Instrumentation: [9aeae2fa] regParam is zero, which might cause numerical instability and overfitting.


In [43]:
test_results = lr_model.evaluate(test_data)

In [44]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|0.20398804024870287|
|-3.8882164596740836|
|-21.753398040287152|
| -7.336155576172587|
|  3.947505509773805|
|  3.996925784062455|
| -4.767843266001819|
| -3.707146048946754|
| -5.308510324314852|
|  18.21834004057746|
|-0.7847468689864172|
|-26.254861549214297|
| -6.245422862316843|
|-17.729631588894392|
|-2.2313074598125127|
| -9.045816506045583|
|  8.234226003117612|
|-1.8143731598980821|
| 12.249913445004097|
|  5.640270528386168|
+-------------------+
only showing top 20 rows



In [45]:
test_results.rootMeanSquaredError

11.298123960065201

In [47]:
test_results.r2 # difference between test value and predicted value

0.9797093653696552

In [48]:
unlabeled_data = test_data.select('features')

In [49]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7242025238451...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.9262720263601...|
|[31.9480174211613...|
+--------------------+
only showing top 20 rows



In [50]:
predictions = lr_model.transform(unlabeled_data)

In [51]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.86042571781695|
|[30.8794843441274...|494.09481644452876|
|[31.1239743499119...| 508.7004518800529|
|[31.1280900496166...| 564.5888423232273|
|[31.3584771924370...| 491.2284449397016|
|[31.3662121671876...|426.59195677242246|
|[31.5147378578019...|494.58033126246323|
|[31.5171218025062...|279.62556669933247|
|[31.5257524169682...|449.27413713419674|
|[31.6005122003032...| 460.9545114505195|
|[31.6610498227460...|417.14310044888725|
|[31.6739155032749...| 501.9799294590955|
|[31.7242025238451...|509.63331015027734|
|[31.8164283341993...| 518.8521230925508|
|[31.8186165667690...|448.64998082994816|
|[31.8279790554652...| 449.0485640529871|
|[31.8512531286083...| 464.7580206636808|
|[31.8627411090001...| 558.1125143339448|
|[31.9262720263601...| 379.9550199993223|
|[31.9480174211613...|456.28060636451164|
+--------------------+------------